In [1]:
# Reducing the dataset to 15,000 images 
# Taking a random 150 immages from both folders and their corresponding variations 
import numpy as np
import pandas as pd
import shutil
import os
import random
from sklearn.model_selection import train_test_split

clear_path = '/kaggle/input/art-images-clear-and-distorted/Art_Dataset_Clear/'
foreign_images = os.listdir(os.path.join(clear_path, 'Foreign'))
indian_images = os.listdir(os.path.join(clear_path, 'Indian'))

# Getting a random 150 images  
foreign_images_subset = random.sample(foreign_images, 150)
indian_images_subset = random.sample(indian_images, 150)

# Splitting into training and testing sets (80% training, 20% testing)
# Important to note that these just contain the name of the files not the actaul image data 
foreign_train, foreign_test = train_test_split(foreign_images_subset, test_size=0.2, random_state=45)
indian_train, indian_test = train_test_split(indian_images_subset, test_size=0.2, random_state=45)

train_images = []
test_images = []

train_images = foreign_train + indian_train
test_images = foreign_test + indian_test

# Matching up the image to its distorted versions 
distorted_path = '/kaggle/input/art-images-clear-and-distorted/Art_Dataset_Distorted/'
distorted_images = os.listdir(os.path.join(distorted_path, 'Distorted'))

def get_distorted_variations(image_name, distorted_folder):
    base_name = os.path.splitext(image_name)[0]
    variations = [f for f in os.listdir(os.path.join(distorted_path, 'Distorted')) if f.startswith(base_name)]
    return variations

In [2]:
# Fetching distorted variations for training and testing
distorted_train = []
for image in train_images:
    distorted_train.extend(get_distorted_variations(image, distorted_path))

distorted_test = []
for image in test_images:
    distorted_test.extend(get_distorted_variations(image, distorted_path))

In [3]:
# Saving the images
output_dir = '/kaggle/working/images_subset'
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'test')
distorted_train_dir = os.path.join(output_dir, 'distorted_train')
distorted_test_dir = os.path.join(output_dir, 'distorted_test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(distorted_train_dir, exist_ok=True)
os.makedirs(distorted_test_dir, exist_ok=True)

In [4]:
# Copy training images to the train directory
for img in train_images:
    if "F" in img:
        shutil.copy(os.path.join(clear_path, 'Foreign', img), train_dir)
    else:
        shutil.copy(os.path.join(clear_path, 'Indian', img), train_dir)
        
for img in test_images:
    if "F" in img:
        shutil.copy(os.path.join(clear_path, 'Foreign', img), test_dir)
    else:
        shutil.copy(os.path.join(clear_path, 'Indian', img), test_dir)
        
for img in distorted_train:
    shutil.copy(os.path.join(distorted_path, 'Distorted', img), distorted_train_dir)

for img in distorted_test:
    shutil.copy(os.path.join(distorted_path, 'Distorted', img), distorted_test_dir)

In [5]:
# Create a zip file
shutil.make_archive('/kaggle/working/images_subset', 'zip', '/kaggle/working', 'images_subset')

'/kaggle/working/images_subset.zip'

In [6]:
import os
import tensorflow as tf

distorted_train_dir = '/kaggle/input/images-15000/images_subset/distorted_train/'
clear_train_dir = '/kaggle/input/images-15000/images_subset/train/'
distorted_test_dir = '/kaggle/input/images-15000/images_subset/distorted_test/'
clear_test_dir = '/kaggle/input/images-15000/images_subset/test/'

def preprocess_image(image_path, target_size=(256, 256)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, target_size)
    image = image / 255.0  # Normalize
    return image

# Using np arrays takes up too much RAM
# Converting np array to tf.data.Dataset will be more efficent 
def load_image_dataset(X, y):
    distorted_paths = [os.path.join(X, fname) for fname in os.listdir(X)]
    
    # The number of clear images has to match the distorted ones
    clear_paths = []
    for fname in os.listdir(y):
        clear_paths.extend([os.path.join(y, fname)] * 50)
        
    dataset = tf.data.Dataset.from_tensor_slices((distorted_paths, clear_paths))
    dataset = dataset.map(lambda distorted, clear: (preprocess_image(distorted), preprocess_image(clear)), num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

In [7]:
# Load training and testing datasets
train = load_image_dataset(distorted_train_dir, clear_train_dir)
test = load_image_dataset(distorted_test_dir, clear_test_dir)
train = train.batch(32)
test = test.batch(32)

In [8]:
# Buidling the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import layers, models

def build_cnn_model(input_shape=(512, 512, 3)):
    model = Sequential()
    # Extracts features from the input images by applying filters 
    # We do multiple filters to extract more details 
    model.add(Conv2D(64, (3,3), 1, activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D()) # reduces the spatial size

    model.add(layers.Conv2D(128, (3, 3), 1,activation='relu', padding='same'))
    model.add(layers.MaxPooling2D())

    model.add(layers.Conv2D(256, (3, 3), 1, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D())
    
    # reconstructs the image back to its original 
    model.add(layers.Conv2DTranspose(256, (3, 3), 1, activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    
    model.add(layers.Conv2DTranspose(128, (3, 3), 1, activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))

    model.add(layers.Conv2DTranspose(64, (3, 3), 1, activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    
    # output layer
    # 3 filters because RGB values 
    model.add(layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same'))

    # decoder network should be for denoising
    # model.add(layers.Conv2DTranspose(512, (3, 3), activation='relu', padding='same'))
    # model.add(layers.UpSampling2D((2, 2)))
    
    return model

In [9]:
# we are using adam because its efficient and has little memory requirement
input_shape = (512, 512, 3)  # Adjust this based on your image size
model = build_cnn_model(input_shape)
model.compile('adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 512, 512, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 128, 128, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 64, 64, 256)    │       590,080 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 128, 128, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 128, 128, 128)  │       295,040 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 256, 256, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 256, 256, 64)   │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 512, 512, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 512, 512, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,331,459 (5.08 MB)

 Trainable params: 1,331,459 (5.08 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Training the data 
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

hist = model.fit(train, epochs=10, batch_size=32)

Epoch 1/10


I0000 00:00:1729394161.340870      63 service.cc:145] XLA service 0x7e703c0050a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729394161.340967      63 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  1/375 ━━━━━━━━━━━━━━━━━━━━ 1:48:04 17s/step - accuracy: 0.2667 - loss: 0.1259

I0000 00:00:1729394175.368152      63 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


375/375 ━━━━━━━━━━━━━━━━━━━━ 104s 231ms/step - accuracy: 0.6524 - loss: 0.0809
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 76s 201ms/step - accuracy: 0.6706 - loss: 0.0803
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 76s 201ms/step - accuracy: 0.6710 - loss: 0.0803
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 76s 202ms/step - accuracy: 0.6710 - loss: 0.0799
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 76s 202ms/step - accuracy: 0.6710 - loss: 0.0799
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 76s 203ms/step - accuracy: 0.6710 - loss: 0.0798
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 77s 205ms/step - accuracy: 0.6710 - loss: 0.0798
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 77s 206ms/step - accuracy: 0.6710 - loss: 0.0798
Epoch 9/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 82s 218ms/step - accuracy: 0.6710 - loss: 0.0798
Epoch 10/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 100s 266ms/step - accuracy: 0.6710 - loss: 0.0798


In [11]:
# Visulizations and Predictions
